In [1]:
0.5

0.5

In [2]:
%pip install torchsummary

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torchsummary

In [2]:
import time
from IPython.display import Audio
import logging

from Freesound.data import *
from Freesound.utils import *
from Freesound.model import *
from Freesound.augmentations import *

import numpy as np
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import torch
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import CosineAnnealingLR, CyclicLR
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms

%reload_ext autoreload
%autoreload 2

seed_everything(0)
logging.basicConfig(level=logging.DEBUG, filename="logs/logs.log", filemode="w+")

In [3]:
names, y_curated_np, y_noisy_np, train_curated, train_noisy, _, labels = get_tables()

#train_audio = load_audio(1, False, 'Freesound/pickles/train_audio_44100.pkl')
#train_audio.pop(534)
y_curated_np = np.delete(y_curated_np, 534, axis=0)
train_curated.drop(534, axis=0, inplace=True)
#train_audio = np.array(train_audio)

mskf = MultilabelStratifiedKFold(n_splits=5, random_state=0)
train_index, test_index = next(mskf.split(y_curated_np, y_curated_np))
#X_train, X_val = train_audio[train_index], train_audio[test_index]
y_train, y_val = y_curated_np[train_index], y_curated_np[test_index]

In [4]:
train_transform = transforms.Compose([
    #RandomParameter(VTLP, [[0.8, 1.2]], p=0.5),
    #MinMaxChunkScaler(),
    #Normalize(),
    RandomParameter(RandomNoise, [[0.01, 0.5]]),
    RandomParameter(Shift, [[2000, 16000]]),
    RandomParameter(TimeStretch, [[0.6, 1.8]]),
    RandomParameter(PitchShift, [[1, 16]]),
    RandomParameter(Distortion, [[-1, -0.3], [.3, 1.]]),
    ToMellSpec(),
    #GetMFCC(),
    PadOrClip(600),
    Normalize(),
    ToTensor(),
    #transforms.ToTensor(),
]) 

In [7]:
def flatten(tensor):
    return tensor.view(256, 256).permute(1, 0)

In [8]:
post_transform = transforms.Compose([
    #ToMellSpec(n_mels=331),
    librosa.power_to_db,
    #GetMFCC(),
    PadOrClip(256),
    Normalize_predef(-24.42848536693399, 13.524654994769366),
    #Normalize(),
    ToTensor(),
    #transforms.ToTensor(),
    flatten,
])

In [9]:
from Freesound.jasper import TinyJasper

In [10]:
##train
hm = Train(model_path="models/tine_jasper_RT.pt", gradient_acumulation=set([100, 150, 200, 250, 300]))

#train_dataset_raw = Dataset_Train(X_train, y_train, preload=True)
train_dataset = Dataset_Train_Preload(y_train, "../data/raw_mels_augs_RT/")
train_dataset_mixmatch = Dataset_Train_Mixmatch(train_dataset, post_transform=post_transform, pad_lenth=256)

#valid_dataset_raw = Dataset_Train(X_val, y_val, post_transform, preload=True)
valid_dataset = Dataset_Train_Preload(y_val, "../data/raw_mels_augs_RT_val/", transforms=post_transform)

train_loader = DataLoader(train_dataset_mixmatch, batch_size=350, num_workers=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, num_workers=16, batch_size=300, shuffle=False)

#model = Kaggle2(80).cuda()
model = TinyJasper(256).cuda()

model_dst = torch.nn.DataParallel(model, device_ids=[0, 1]).cuda()
criterion = nn.BCEWithLogitsLoss().cuda()

optimizer = Adam(params=model.parameters(), lr=1e-4)

In [ ]:
hm.fit(train_loader, valid_loader, model_dst, criterion, optimizer, None, epoches=1000)